This notebook assumes you have some version of Python >= 3.10 installed. If not, see https://www.python.org/downloads/ and download it according to your device type. You can check using the cell below.

In [ ]:
import sys
print(sys.version)

We'll create a new Python environment for this project to avoid any dependency conflicts with other projects on your device.

In [ ]:
import os
import sys
import shutil
import subprocess

env_name = "mediapipe_env"

# -----------------------------
# Remove old env (optional)
# -----------------------------
if os.path.exists(env_name):
    print("Removing old environment...")
    shutil.rmtree(env_name)

# -----------------------------
# Find latest Python version
# -----------------------------
def find_latest_python():
    candidates = ["python3.14", "python3.13", "python3.12", "python3.11", "python3.10", "python3", "python"]
    for cmd in candidates:
        try:
            subprocess.run(cmd.split() + ["--version"], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            return cmd.split()
        except:
            continue
    return [sys.executable]  # fallback to current Python

python_cmd = find_latest_python()
print("Using Python:", " ".join(python_cmd))

# -----------------------------
# Create virtual environment
# -----------------------------
subprocess.run(python_cmd + ["-m", "venv", env_name], check=True)

# -----------------------------
# Determine OS paths
# -----------------------------
if os.name == "nt":  # Windows
    pip_path = os.path.join(env_name, "Scripts", "pip")
    python_path = os.path.join(env_name, "Scripts", "python")
else:  # macOS / Linux
    pip_path = os.path.join(env_name, "bin", "pip")
    python_path = os.path.join(env_name, "bin", "python")

# -----------------------------
# Install dependencies
# -----------------------------
subprocess.run([pip_path, "install", "--upgrade", "pip"])
subprocess.run([pip_path, "install", "mediapipe", "opencv-python", "ipykernel"])

# -----------------------------
# Register kernel
# -----------------------------
subprocess.run([
    python_path,
    "-m",
    "ipykernel",
    "install",
    "--user",
    "--name",
    env_name,
    "--display-name",
    "mediapipe_env"
])

print("\nEnvironment setup complete.")
print("➡ Restart Jupyter and select: mediapipe_env")

Change the Python kernel to `mediapipe_env` in the top right. If it isn't showing up, restart the IDE (VSCode, Cursor, etc.) you are using. 

Then restart the kernel and run the cells below. 

**You do NOT need to run the above cells above again!**

In [ ]:
from mediapipe.tasks import python
from mediapipe.tasks.python.vision import FaceLandmarker, FaceLandmarkerOptions, RunningMode
import mediapipe as mp
import cv2
import numpy as np
import urllib.request
import os

# Download model if needed
if not os.path.exists("face_landmarker.task"):
    print("Downloading model...")
    urllib.request.urlretrieve(
        "https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/latest/face_landmarker.task",
        "face_landmarker.task"
    )
    print("Done.")
else:
    print("Model already downloaded.")

# -----------------------------------------------
# Key landmark indices (from MediaPipe 478-point map)
# -----------------------------------------------
LANDMARKS = {
    "left_eye_inner":   133,
    "left_eye_outer":   33,
    "right_eye_inner":  362,
    "right_eye_outer":  263,
    "left_eye_top":     159,
    "left_eye_bottom":  145,
    "right_eye_top":    386,
    "right_eye_bottom": 374,
    "nose_tip":         4,
    "nose_bottom":      2,
    "mouth_left":       61,
    "mouth_right":      291,
    "mouth_top":        13,
    "mouth_bottom":     14,
    "left_eyebrow_inner":  107,
    "right_eyebrow_inner": 336,
    "chin":             152,
    "forehead":         10,
}

# -----------------------------------------------
# Define which distances to measure
# -----------------------------------------------
MEASUREMENTS = {
    "Eye Distance":        ("left_eye_outer",     "right_eye_outer"),
    "Left Eye Width":      ("left_eye_inner",      "left_eye_outer"),
    "Right Eye Width":     ("right_eye_inner",     "right_eye_outer"),
    "Left Eye Height":     ("left_eye_top",        "left_eye_bottom"),
    "Right Eye Height":    ("right_eye_top",       "right_eye_bottom"),
    "Mouth Width":         ("mouth_left",          "mouth_right"),
    "Mouth Height":        ("mouth_top",           "mouth_bottom"),
    "Nose to Chin":        ("nose_tip",            "chin"),
    "Face Height":         ("forehead",            "chin"),
    "Eyebrow Gap":         ("left_eyebrow_inner",  "right_eyebrow_inner"),
    "Left Eye-Brow Gap":   ("left_eye_top",        "left_eyebrow_inner"),
    "Right Eye-Brow Gap":  ("right_eye_top",       "right_eyebrow_inner"),
}

def get_pixel_coords(landmark, frame_w, frame_h):
    return np.array([landmark.x * frame_w, landmark.y * frame_h])

def get_normalized_coords(landmark):
    """Use normalized coords (0-1) for ratios, z included for 3D distance."""
    return np.array([landmark.x, landmark.y, landmark.z])

def pixel_distance(p1, p2):
    return np.linalg.norm(p1 - p2)

def draw_measurement(frame, p1, p2, label, value, color=(0, 255, 255)):
    cv2.line(frame, tuple(p1.astype(int)), tuple(p2.astype(int)), color, 1)
    mid = ((p1 + p2) / 2).astype(int)
    cv2.putText(frame, f"{label}: {value:.1f}px", (mid[0] + 5, mid[1]),
                cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1)

# -----------------------------------------------
# Main loop
# -----------------------------------------------
base_options = python.BaseOptions(model_asset_path="face_landmarker.task")
options = FaceLandmarkerOptions(
    base_options=base_options,
    running_mode=RunningMode.IMAGE,
    num_faces=1
)

cap = cv2.VideoCapture(0)
h, w = None, None

with FaceLandmarker.create_from_options(options) as detector:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        h, w = frame.shape[:2]

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
        results = detector.detect(mp_image)

        if results.face_landmarks:
            landmarks = results.face_landmarks[0]

            # Draw all landmark dots
            for lm in landmarks:
                cx, cy = int(lm.x * w), int(lm.y * h)
                cv2.circle(frame, (cx, cy), 1, (180, 180, 180), -1)

            # Highlight key landmarks
            for name, idx in LANDMARKS.items():
                lm = landmarks[idx]
                cx, cy = int(lm.x * w), int(lm.y * h)
                cv2.circle(frame, (cx, cy), 3, (0, 200, 255), -1)

            # Calculate and draw measurements
            distances = {}
            for label, (a, b) in MEASUREMENTS.items():
                idx_a = LANDMARKS[a]
                idx_b = LANDMARKS[b]
                p1 = get_pixel_coords(landmarks[idx_a], w, h)
                p2 = get_pixel_coords(landmarks[idx_b], w, h)
                dist = pixel_distance(p1, p2)
                distances[label] = dist
                draw_measurement(frame, p1, p2, label, dist)

            # -----------------------------------------------
            # Compute useful ratios (scale-independent)
            # -----------------------------------------------
            eye_dist = distances["Eye Distance"]
            if eye_dist > 0:
                ratios = {
                    "Mouth/Eye ratio":    distances["Mouth Width"] / eye_dist,
                    "L Eye aspect ratio": distances["Left Eye Height"] / max(distances["Left Eye Width"], 1),
                    "R Eye aspect ratio": distances["Right Eye Height"] / max(distances["Right Eye Width"], 1),
                    "Face aspect ratio":  distances["Face Height"] / max(eye_dist, 1),
                }

                # Display ratios in top-left panel
                cv2.rectangle(frame, (0, 0), (220, 20 + len(ratios) * 20), (0, 0, 0), -1)
                for i, (label, val) in enumerate(ratios.items()):
                    cv2.putText(frame, f"{label}: {val:.2f}", (5, 18 + i * 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 180), 1)

        cv2.imshow("Face Measurements", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()